In [1]:
import sys
import copy
import random
import hashlib
# import cupy as cp 
import numpy as np
from math import pi
import numpy.linalg as LA

from os.path import abspath, join
sys.path.append(abspath(join('..')))
sys.path.append(abspath(join('../..')))

from datetime import datetime
from os.path import expanduser

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from LevelSetPy.Grids import *
from LevelSetPy.Utilities import *
from LevelSetPy.DDPReach import *
from LevelSetPy.Visualization import *
from LevelSetPy.DynamicalSystems import *
from LevelSetPy.BoundaryCondition import *
from LevelSetPy.InitialConditions import *
from LevelSetPy.SpatialDerivative import *
from LevelSetPy.ExplicitIntegration import *
from BRATVisualization.rcbrt_visu import RCBRTVisualizer

%matplotlib inline

In [2]:
args = Bundle(dict(visualize=True, flock_num=4, resume="murmurations_flock_01_02-06-22_17-43.hdf5", flock_payoff=False, pause_time=1e-5))

In [3]:
grid_min = expand(np.array((-.75, -1.25, -np.pi)), ax = 1)
grid_max = expand(np.array((3.25, 1.25, np.pi)), ax = 1)
pdDims = 2                      # 3rd dimension is periodic
resolution = 100
N = np.array(([[
                resolution,
                np.ceil(resolution*(grid_max[1, 0] - grid_min[1, 0])/ \
                            (grid_max[0, 0] - grid_min[0, 0])),
                resolution-1
                ]])).T.astype(int)
grid_max[2, 0] *= (1-2/N[2,0])
g = createGrid(grid_min, grid_max, N, pdDims)

In [4]:
class VarHJIApprox(RocketSystem):
    def __init__(self, eta=.5, rho=.99, dx=.1, grid=None, X=None):
        super().__init__()
        """
            This class uses iterative dynamic game to compute the level sets of the
            value function for all scheduled trajectories of a dynamical system. For
            details see the following bibliography:

            "A Second-Order  Reachable Sets Computation Scheme via a  Cauchy-Type
            Variational Hamilton-Jacobi-Isaacs Equation." IEEE Letter to Control
            Systems Society, by Lekan Molu, Ian Abraham, and Sylvia Herbert.

            Parameters:
            -----------
            Cost improvement params:
                .eta: Stopping condition for backward pass;
                .rho: Regularization condition.
                .dx: delta between every discretization between trajectories that emerge on 
                    the state space. 
                .grid: Optional. Grid properties as already created by createGrid in the LevelSetPy Toolbox.
            
            .X: All initial state trajectories that emanate from the state space.
        """
        self.eta = eta
        self.rho = rho

        self.X = X if X else  self.generate_init_conds(dx, grid)


        self.system = RocketSystem()  

        # Buffer for all values
        self.value_buf = list()

    def generate_init_conds(self, dx = .1, grid=None):        
        """        
            This method accumulates all initial trajectories within the
            given bounds of the state space.

            Inputs
            ------
            .dx: delta between every discretization between trajectories that emerge on 
                the state space. 
            .grid: Optional. Grid properties as already created by createGrid in the LevelSetPy Toolbox.
            
            Output
            -------
            .X: A Numpy array of all trajectories that emanate as initial conditions from 
            the state space.
        """
        # contants
        a = 64 # ft/sec^2
        g = 32 # ft/sec^2
        dx = .1  # state space resolution

        if grid is not None:
            grid_min  = grid.min
            grid_max  = grid.max
        else:
            grid_min = np.array(([0,0, 64, g]))
            grid_max = np.array(([1e5, 1e5, 64, g+20]))

        X = [grid_min]

        while np.any(X[-1]<grid_max):
            X += [X[-1]+dx]

        X = np.array(X)

        # # fix the inconsistencies in (x1, x2, x3, x4)
        indices = [np.nan for idx in range(X.shape[0])]
        for dim in range(X.shape[-1]):
            indices[dim] = X[:,dim]>grid_max[dim]

            # replace trajectories with bounds that exceed max 
            # values along respective dimensions
            X[indices[dim],dim] = grid_max[dim]   

            
        return X


In [5]:
hji  = VarHJIApprox()

In [6]:
X = hji.X
X.shape

(1000001, 4)

$\pi_{i}$,$\pi_{r_i}$, $u_{r_i}$, $v_{r_i}$, $x_{r_i}$, $\dot{x}_{r_i}$, $u_i$, $v_i$, $u^\ast$, $v^\ast$

In [7]:
def backward(𝑥𝑖, stop_cond=None):
    """
    Input: 
        .𝑥𝑟𝑖 - Given initial condition for trajectory (state)
        .𝜋𝑟𝑖: A tuple of nominal maximizing (u) and 
                minimizing (v) controllers with:
            .𝑢𝑖 - Control input (evader).
            .𝑣𝑖 - Disturbance input (pursuer).
    Output:
        .𝜋𝑖 : A tuple of nominal maximizing (u) and 
                minimizing (v) controllers with:
            .𝑢𝑖 - Global Control input (evader).
            .𝑣𝑖 - Global Disturbance input (pursuer).
    """
    #𝑢𝑟𝑖, 𝑣𝑟𝑖 = 𝜋𝑟𝑖 
    for T in range(hji.system.T-1, -1, -1):
    # for T in range(120, -1, -1):
        𝑢𝑟𝑖 = 1.6-1.58*(T/100.)
        𝑣𝑟𝑖 = 1.6+1.58*(T/100.)
        
        # generate \dot{𝑥}𝑟𝑖 from dynamics
        dxri = hji.system.dynamics(𝑥𝑖, (𝑢𝑟𝑖, 𝑣𝑟𝑖))        
        #print(dxri)
        
        # do $Runge Kutta
        xri = hji.system.rk4_integration(dxri, (𝑢𝑟𝑖, 𝑣𝑟𝑖))
        print(T, "|", xri)
        
        # compute the value and co-state
        hji.system.value_funcs(xri, t=T)
        # compute the Hamiltonians w.r.t u
        hji.system.hamiltonian(xri, (𝑢𝑟𝑖, 𝑣𝑟𝑖), hji.system.Vx[T], t=T)
        
        # calculate 𝑢∗, 𝑣∗
        

        if T < 90: break
    # return xi, pol_i  
            
backward(X[0])


99 | [7740.55825894  256.24745251 1407.36901414   41.11203296]
98 | [7738.80004659  256.19609165 1407.04667521   41.1026168 ]
97 | [7735.125937    256.08876364 1406.37308845   41.08294   ]
96 | [7729.53684738  255.92549528 1405.34842202   41.05300747]
95 | [7722.03417294  255.70632731 1403.97293171   41.01282667]
94 | [7712.61978663  255.43131447 1402.24696088   40.96240765]
93 | [7701.29603859  255.10052539 1400.17094041   40.90176299]
92 | [7688.06575564  254.71404265 1397.74538853   40.83090782]
91 | [7672.9322405   254.27196273 1394.97091076   40.74985983]
90 | [7655.89927103  253.774396   1391.84819969   40.65863927]
89 | [7636.97109926  253.22146666 1388.37803486   40.55726889]


### Two-Player Differential Game Between Two Rockets

At issue are two rocket missiles on a state space: (i) a pursuer rocket $P$ whose goal is to "capture" an "evader" rocket $E$ within a given time interval $[-T, 0]$ for $T>0$. This is akin to a war game where e.g. an iron dome must intercept a missile within a time period. Outside the _measure_  $T$, the evading rocket may land in a large civilian population and destroy a lot of properties. The pursuer is seeking to minimize the time of capture, while the evader is seeking to maximize the time of capture.

In what follows, we shall set up a two-player differential game system of equations between these two agents in absolute coordinates. We leverage Dreyfus' rocket launch example used by S.K. Mitter in his [Automatica paper from 1966](https://scholar.google.com/scholar_url?url=https://stuff.mit.edu/people/mitter/publications/1_successive_approx_AUTO.pdf&hl=en&sa=T&oi=gsb-gga&ct=res&cd=0&d=9474146906219145202&ei=lqGDYqvgNM6Uy9YPjP-C6A4&scisig=AAGBfm0MFGEhs6vvhQeFLR1e3EQvtwya3g).

Here are the key variables for the pursuer and evader respectively:


| $y_1/y_5$ (ft)     | $y_2/y_6$  (ft)    | $y_3/y_7$  (ft/sec)    |     $y_4/y_8$  (ft/sec) |
| :--:     | :---:               | :---:               |  :---:               | 
| Horizontal range    |      Altitude   |    Horizontal velocity    | Vertical velocity  |

The equations of motion are adopted from Dreyfus' construction as follows:

$$
\begin{align}
\begin{array}{cc cc}
    &\dot{y}_1 = y_3,          &\dot{y}_5 = y_7, \\
    &\dot{y}_2 = y_4,          &\dot{y}_6 = y_8, \\
    &\dot{y}_3 = a\cos(u),     &\dot{y}_7 = a\cos(v), \\
    &\dot{y}_4 = a\sin(u) - g, &\dot{y}_8 = a\sin(v) - g.
\end{array}
\end{align}
$$

where $u(t), t \in [-T,0]$ is the controller under the coercion of the evader and $v(t), t \in [-T,0]$ is the controller under the coercion of the pursuer i.e. the pursuer is minimizing while the evader is maximizing. The full state dynamics is given by

$$
\begin{align}
 \dot{y} = \left(\begin{array}{c}
                  \dot{y}_1 \\
                  \dot{y}_2 \\
                  \dot{y}_3 \\
                  \dot{y}_4 \\
                  \dot{y}_5 \\
                  \dot{y}_6 \\
                  \dot{y}_7 \\
                  \dot{y}_8 \\
 \end{array}\right)
 =
 \left(\begin{array}{c}
 y_3 \\
 y_4 \\
 a\cos(u) \\
 a\sin(u) - g \\
 y_7 \\
 y_8 \\
 a\cos(v) \\
 a\sin(v) - g
 \end{array}\right).
\end{align}
$$

In relative coordinates between the two rockets, we write:

$x = \left(\begin{array} y_1 - y_5 \\ y_2 - y_6 \\ y_3 - y_7 \\ y_4 - y_8  \end{array}\right) \implies \dot{x} = \left(\begin{array} \dot{x}_1 \\ \dot{x}_2 \\ \dot{x}_3 \\ \dot{x}_4  \end{array}\right) =  \left(\begin{array} \dot{y}_1 - \dot{y}_5 \\ \dot{y}_2 - \dot{y}_6 \\ \dot{y}_3 - \dot{y}_7 \\ \dot{y}_4 - \dot{y}_8  \end{array}\right)$.

| $x_1$ (ft)     | $x_2$  (ft)    | $x_3$  (ft/sec)    |     $x_4$  (ft/sec) |
| :--:     | :---:               | :---:               |  :---:               | 
| Relative horizontal distance    |      Relative vertical distance   |    Resultant horizontal velocity   | Resultant velocity velocity.  |

So that,

$$
\begin{align}
    &\dot{x}_1 = x_3,\\
    &\dot{x}_2 = x_4,\\
    &\dot{x}_3 = a\cos(u) - a\cos(v),\\
    &\dot{x}_4 = a\sin(u) - a\sin(v)
\end{align}
$$

### Target Set Construction

Encode the Target Set as a minimum radius of capture between the two rockets.

We want to construct a target set such that the [Reduced Vertical Separation Minimum](https://www.faa.gov/air_traffic/separation_standards/rvsm/) between the two rockets at any time $t \in [-T, 0]$ is at least $w=1000 ft$. The RVSM we use is consistent with FAA recommendations for airborne systems. Capture occurs when the $l_2$ distance between the two missiles below the specified range given. We write the target set thus:  
 
$$
 \mathcal{T}  = \{x \in \mathcal{X} \mid \sqrt{x_1^2 + x_2^2} \le r \} := \{x \in \mathcal{X} \mid V(x, 0) \le 0 \}
$$

i.e. the _**zero-level set**_ $V(x,0)$ from $\mathcal{T}$'s definition:

$$
\begin{align}
     V(x,0) = \sqrt{x_1^2 + x_2^2} - r.
\end{align}
$$

We can write the _**robustly controlled backward reachable tube**_ for a value function $V: \mathcal{X}\times [-T,0] \rightarrow \mathbb{R}$ as 

$$
\begin{align}
 \ell([-T,0], \mathcal{T}) &=  \{x \in \mathcal{X}, t \in [-T, 0] \mid V(x, t) \le 0 \}.
 \end{align}
$$

At time $t \in [-T,0]$, the variational HJI PDE is 

$$
\begin{align}
     \dfrac{\partial V(x,t)}{\partial t} + \min \left\{0, \max_{u} \min_{v} H\left(x,u,v,\dfrac{\partial V(x,t)}{\partial x} \right)\right\} = 0.
\end{align}
$$

Whereupon the Hamiltonian is given by 

$$
\begin{align}
H(x,p) &= \langle V_x, f(x,u,v) \rangle\\
 &= \left(\begin{array}{cc cc cc cc}p_1 & p_2 & p_3 & p_4 \end{array}\right) \left(\begin{array}{c}
 x_3 \\
 x_4 \\
 a\cos(u) - a\cos(v) \\
 a\sin(u) - a\sin(v) 
 \end{array}\right)
 \\
H(x,p) &=  p_1x_3 + p_2x_4 + p_3a(\cos(u) - \cos(v)) + p_4a(\sin(u) - \sin(v))
\end{align}
$$

<!-- Or 
$$
\begin{align}
H(t; x,u,v,p) &= \dfrac{2 x_1}{\sqrt{x_1^2 + x_5^2}} \dot{x}_1 + \dfrac{2 x_2}{\sqrt{x_2^2 + x_6^2}} \dot{x}_2 
\\
&= \dfrac{2 x_1}{\sqrt{x_1^2 + x_5^2}} x_3 + \dfrac{2 x_2}{\sqrt{x_2^2 + x_6^2}} x_4.
\end{align} 
$$ -->

The derivative of the Hamiltonian is 
$$
\begin{align}
&H_u = -p_3a\sin(u) + p_4a\cos(u) \\
&H_{uu} = -p_3a\cos(u) - p_4a\sin(u) \\
&H_{uv} = \textbf{0}_{4\times4}  \\
&H_{ux} = \textbf{0}_{4\times4} 
\end{align}
$$
Partial derivative of $H$ with respect to $v$
$$
\begin{align}
&H_v = p_3a\sin(v) - p_4a\cos(v) \\
&H_{vv} = p_3a\cos(v) + p_4a\sin(v) \\
&H_{vu} = \textbf{0}_{4\times4} \\
&H_{vx} = \textbf{0}_{4\times4} 
\end{align}
$$

Partial derivative of $H$ with respect to $x$
$$
\begin{align}
&H_x = \begin{bmatrix}0 &0 &p_1 &p_2 \end{bmatrix}^T \\
&H_{xx} = 0_{4\times4} \\
&H_{xu} = \textbf{0}_{4\times4} \\
&H_{xv} = \textbf{0}_{4\times4} 
\end{align}
$$

### Value function derivatives


$$
\begin{align}
     V(x,0) &= \sqrt{x_1^2 + x_2^2} - r; \qquad V_x = \dfrac{2 x_1}{\sqrt{x_1^2 + x_2^2}}{x}_3 + \dfrac{2 x_2}{\sqrt{x_1^2 + x_2^2}}{x}_4; \\
     V_{xx} &= \dfrac{2 \left[x_1^3+x_2^2(x_2+x_3)+x_1 x_2 (x_2 -x_3 - x_4) + x_1^2(x_2+x_4)\right]}{\left(\sqrt{x1^2 + x2^2}\right)^3}
\end{align}
$$

In [ ]:
def get_all_trajs():
    
    """        
        What we do here is really simple. We run as many 
        a fwd and backward pass of ddp algos as possible  
        on trajectories that are finely spaced apart by g.dx 

        This part accumulates all trajectories within grid bounds
    """
    # contants
    a = 64 # ft/sec^2
    g = 32 # ft/sec^2
    dx = .1  # state space resolution
    resolution_size = 100

    grid_min = np.array(([0,0, 64, g]))
    grid_max = np.array(([1e5, 1e5, 64, g+20]))

    X = [grid_min]

    while np.any(X[-1]<grid_max):
        X += [X[-1]+dx]

    X = np.array(X)



    # # fix the inconsistencies in (x1, x2, x3, x4)
    indices = [np.nan for idx in range(X.shape[0])]
    for dim in range(X.shape[-1]):
        indices[dim] = X[:,dim]>grid_max[dim]

        # replace trajectories with bounds that exceed max 
        # values along respective dimensions
        X[indices[dim],dim] = grid_max[dim]   

        
    return X

In [ ]:
def backward(xdot, u_r, v_r):
    '''
    input: 
        x_r - trajectory of the nominal states
        u_r - trajectory of the nominal control (evader)
        v_r - trajectory of the nominal disturbance (pursuer)
        Val_opt - optimal value function
        t_grid - time grid partitions the time interval [-T, 0]
    output:
        Val_opt - updated optimal value function
        Vx_opt - gradient of optimal value function
        Vxx_opt - Hessian matrix of optimal value function
        u_opt - updated optimal control 
        v_opt - updated optimal disturbance
    '''
    xr = RK4(xdot)
    ur = 1.6 - 1.5*(t/100)
    vr = -ur
    for t in range(0, -T, -1):
        
        # compute Vr(xr, t) : Note that this is the value for using a nominal control to run xr
        # See equation 36 in paper
        # Solve for rhs of 36 first:
        p = 
        H = hamiltonian(x, u, v, p):
        
        
        
    
    t_eff = t_grid[0]
    eta = 0.1
    
    u_opt = np.zeros(len(t_grid))
    v_opt = np.zeros(len(t_grid))
    
    #initialize Val_opt, Vx_opt, Vxx_opt
    Val_opt = np.zeros(len(t_grid))
    Vx_opt = np.zeros([len(t_grid),4])
    Vxx_opt = np.zeros([len(t_grid),4,4])
    Val_opt[-1], Vx_opt[-1], Vxx_opt[-1] = TargetSetFunc(x_r[-1])
    
    
    # The cost of the trajectory is equal to the value of TargetSetFunc when state first enters the target set 
    Val_r, _, _ = TargetSetFunc(x_r[-1])  # Calculate the terminal cost of the trajectory     
    
    for i in range(len(t_grid)-1,0,-1):
        
        t = t_grid[i]
        
        # calculate the value function of the nominal policy
        Val_t, _, _ = TargetSetFunc(x_r[i])
        if Val_t<=0:
            Val_r = Val_t 
        
        # calculate Val_opt - Val_r 
        Val_imp = Val_opt[i] - Val_r
        alp = 1.0
        
        if abs(Val_imp)<eta:
            tf = t
            t0 = t_grid[i-1]
            X0 = np.concatenate((x_r[i], Val_opt[i], Vx_opt[i],Vxx[i].reshape(4**2)), axis=0)
            X = RK45(hamiltonian_RK45, t0, X, tf, args=(x_r[i], u_r[i], v_r[i], alp))
        else:
            alp = 0.9*alp ## add code for line search
        
        
        x_r[i-1] = X[0:4]
        Val_opt[i-1] = X[4:5]
        Vx_opt[i-1] = X[5:9]
        Vxx_opt[i-1] = X[9:25].reshape((4,4))
        u_opt[i-1] = np.arctan(Vx_opt[i-1,3]/Vx_opt[i-1,2])
        v_opt[i-1] = np.arctan(Vx_opt[i-1,3]/Vx_opt[i-1,2])
        u_opt[i-1] = u_r[i-1] + alp*(u_opt[i-1]-u_r[i-1])
        v_opt[i-1] = v_r[i-1] + alp*(v_opt[i-1]-v_r[i-1])        
    
    return x_r, u_opt, v_opt, Val_opt, Vx_opt, Vxx_opt    
            



In [ ]:
def forward(x_r, u_opt, v_opt, Val_opt, Vx_opt, Vxx_opt, t_grid):
    '''
    input: 
        x_r - trajectory of the nominal states
        u_opt - trajectory of the optimal control (evader)
        v_opt - trajectory of the optimal disturbance (pursuer)
        t_grid - time grid partitions the time interval [-T, 0]
        Val_opt - optimal value function
        Vx_opt - gradient of optimal value function
        Vxx_opt - Hessian matrix of optimal value function
    output:
        Val_opt - updated optimal value function
        x_r - updated trajectory of the state
        u_r - updated trajectory of the nominal control
        v_r - updated trajectory of the nominal disturbance
    '''
    
    delta_x = np.zeros(4)
    
    for i in range(len(t_grid)-1):
                    
        t = t_grid[i]
        dx, f_x, f_u, f_v = system(x_r[i], u_opt[i], v_opt[i])
        H, H_u, H_uu, H_uv, H_ux, H_v, H_vv, H_vx, H_x, H_xx = hamiltonian(x_r[i], u_opt[i], v_opt[i], Vx_opt[i])
        
        H_matrix = np.array([[H_uu, H_uv],[H_vu, H_vv]])
        U = sp.linalg.cholesky(H_matrix)
        L = U.T        
        b = -np.concatenate(H_ux+f_u@Vxx_opt[i], H_vx+f_v@Vxx_opt[i], axis=0)
        k = sp.linalg.solve_triangular(U, sp.linalg.solve_triangular(L, b, lower=True))
                            
        k_u = k[0]
        k_v = k[1]
        
        t0 = t
        tf = t_grid[i+1]
        delta_x = RK45(system_RK45, t0, delta_x, tf, args=(x_r[i], u_r[i], v_r[i], k_u, k_v))
        
        Val_opt[i+1] = Val_opt[i+1] + delta_x@Vx_opt[i+1] + 0.5*delta_x@Vxx_opt[i+1]@delta_x
        
        u_r[i+1] = u_r[i+1] + k_u@delta_x
        v_r[i+1] = v_r[i+1] + k_v@delta_x
        
    return x_r, u_r, v_r, Val_opt
        
    
        
    
    
    
        
        
        
    
    
    

In [ ]:
H = np.zeros((4))

H[:] = 1
H

In [ ]:
np.linspace(-2, 0, 100)